In [1]:
#@title Mount drive

from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

IMAGE_SHAPE = (224, 224)
INPUT_SHAPE = IMAGE_SHAPE + (3,)
AUTO_TUNE = tf.data.experimental.AUTOTUNE


def get_dataset(dataset_path):
    dataset_get = ImageDataGenerator()
    dataset = dataset_get.flow_from_directory(dataset_path, target_size=IMAGE_SHAPE)
    return dataset


def plot_learning_curves(history, history_ft=None):
    if history_ft is not None:
        history.history['accuracy'] += history_ft.history['accuracy']
        history.history['loss'] += history_ft.history['loss']
        history.history['val_accuracy'] += history_ft.history['val_accuracy']
        history.history['val_loss'] += history_ft.history['val_loss']

    plt.subplot(2, 1, 1)
    plt.plot(history.history['accuracy'], label='training')
    plt.plot(history.history['val_accuracy'], label='validation')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    if history_ft is not None:
        plt.plot([history.epoch[-1], history.epoch[-1]], plt.ylim(), label='start fine-tuning')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(history.history['loss'], label='training')
    plt.plot(history.history['val_loss'], label='validation')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    if history_ft is not None:
        plt.plot([history.epoch[-1], history.epoch[-1]], plt.ylim(), label='start fine-tuning')
    plt.legend()
    plt.show()


# prepare paths
dataset_path = Path('/content/gdrive/My Drive/DL/COVID19-project/COVIDx')
model_path = Path('/content/gdrive/My Drive/DL/COVID19-project/models/resnet50v2.h5')
model_path.parent.mkdir(parents=True, exist_ok=True)

# build input pipeline
train_ds = get_dataset(dataset_path / 'train')
val_ds = get_dataset(dataset_path / 'validation')
test_ds = get_dataset(dataset_path / 'test')
n_classes = train_ds.num_classes

# build model
base = ResNet50V2(include_top=False, pooling='avg', input_shape=INPUT_SHAPE)
head = Dense(n_classes)
inputs = Input(shape=INPUT_SHAPE)
x = preprocess_input(inputs)
x = base(x, training=False)
outputs = head(x)
model = Model(inputs, outputs)

# training settings
learning_rate = 0.001
epochs = 20
loss = CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
callbacks = [
    ModelCheckpoint(filepath=model_path, monitor='val_accuracy', mode='max', save_best_only=True, verbose=2),
    # TensorBoard(log_dir=logs_path)
]

# train only head (feature extraction with pre-trained model)
base.trainable = False
model.compile(optimizer=Adam(lr=learning_rate), loss=loss, metrics=metrics)
model.summary()
history = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=callbacks)
plt.figure()
plot_learning_curves(history)

# fine-tune some layers
learning_rate /= 10
epochs += 10
fine_tune_at = 20
base.trainable = True
for layer in base.layers[:fine_tune_at]:
    layer.trainable = False
model.compile(optimizer=Adam(lr=learning_rate), loss=loss, metrics=metrics)
model.summary()
history_ft = model.fit(train_ds, epochs=epochs, initial_epoch=history.epoch[-1]+1, validation_data=val_ds,
                        callbacks=callbacks)
plot_learning_curves(history, history_ft)
model.save(model_path)

# evaluate model
# TODO: night's test should give over-fitting (>90% training accuracy, poor generalization)
# TODO: add confusion matrix and other metrics for evaluation (code below) -> should suck now
# TODO: add data augmentation -> over-fitting should be reduced (i.e. test accuracy should improve)
# TODO: add resampling -> confusion matrix should improve
# TODO: add metric for cross-validation, val_acc is not adequate due to the imbalance (e.g. f1 score, but we have
#  3 classes..., google it)
# TODO: add grad-cam
# TODO: add COVID-Net
# TODO: run tests
# TODO: image saving and model loading

# confusion matrix
# cm = confusion_matrix(test_data_gen.classes, predictions)
# ticks = np.arange(n_classes)
# plt.figure()
# sns.heatmap(cm, annot=True, fmt="d")
# plt.ylabel('Actual label')
# plt.xlabel('Predicted label')
# plt.savefig(os.path.join(args.results, model_name + '_confusion_matrix.png'))
# plt.show()
#
# # precision, recall, f1-score, accuracy, etc.
# cr = classification_report(test_data_gen.classes, predictions, target_names=class_names)
# print('Classification report')
# print(cr)
# with open(os.path.join(args.results, model_name + '_report.txt'), mode='w') as f:
#     f.write(cr)

Found 4796 images belonging to 3 classes.
Found 1976 images belonging to 3 classes.
Found 2325 images belonging to 3 classes.
94674944/94668760 [==============================] - 1s 0us/step
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_BiasAdd (TensorF [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50v2 (Functional)      (None, 2048)              23564800  
_________________________________________________________________
dense (Dense)                (None, 3)                 6147      
Total params: 23,570,947
Trainable params: 6,147
Non-trainabl